# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

As a part of Generative Artifcial Intelligence Nanodegree program by Udacity, this is my first project on "Apply Lightweight Fine-Tuning to a Foundation Model" Lightweight fine-tuning is one of the most important techniques for adapting foundation models, because it allows is to modify foundation models for 
our needs without needing substantial computational resources.

Parameter-efficient fine-tuning is a technique to use pre-trained machine learning models with minimal additional data or computational resources. Basically in this technique, we use specific parameters rather than retraining the entire model. This approach, often used in transfer learning, accelerates convergence and enhances generalization, particularly in NLP, computer vision, and related fields.

In this project, I've applied parameter-efficient fine-tuning using the Hugging Face peft library. In order to do, I've followed following steps.

Step 1: Load a Pre-trained Model and Evaluate Its Performance
I have used thee "distilbert-base-uncased" model from Hugging Face peft library. I  have used the subset of Amazon Polarity dataset. As entire dataset was quite heavy.
In this step, I have prepared the dataset and have evaluated the pre-trained model's performance on the validation dataset.

Step 2: Perform Parameter-Efficient Fine-Tuning Using the Pre-trained Model
In this step, I've defined training arguments for fine-tuning and have also Fine-tuned the pre-trained model on the training subset.
I've also saved the fine-tuned model.

Step 3: Perform Inference Using the Fine-Tuned Model and Compare Its Performance to the Original Model
In this step, I've loaded the fine-tuned model and have evaluated the fine-tuned model's performance on the validation dataset.
I've also compared the trainer evaluation results with the final fine-tuned evaluation results.

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [5]:
#pip install scikit-learn torch transformers datasets 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
# This step is to import libaries
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, load_metric

# This step is to create tokenizer from pre-trained model "distilbert-base-uncased"
foundation_model_is ="distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(foundation_model_is)
foundation_model = AutoModelForSequenceClassification.from_pretrained(foundation_model, num_labels=2)

# This step is to load Amazon Polarity dataset and to create a small subset for training and validation
ds_amazon_polarity = load_dataset("amazon_polarity")
train_dataset = ds_amazon_polarity["train"].shuffle(seed=42).select(range(2000))
val_dataset = ds_amazon_polarity["test"].shuffle(seed=42).select(range(400))

# This step is to Tokenize datasets
def preprocess_function(examples):
    return tokenizer(examples["content"], truncation=True, padding="max_length", max_length=128)

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

metric = load_metric("accuracy", trust_remote_code=True)

def compute_metrics(eval_op):
    predictions = eval_op.predictions.argmax(-1)
    return metric.compute(predictions=predictions, references=eval_op.label_ids)

# This step are to evaluate Foundation model which is "distilbert-base-uncased"
foundation_training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=30,
    logging_dir="./logs",
    logging_steps=10,
)

foundation_trainer = Trainer(
    model=foundation_model,
    args=foundation_training_args,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

# Evaluation results on Foundation model
foundation_evaluation_result = foundation_trainer.evaluate()
print(f"Evaluation results on Trainer: {foundation_evaluation_result}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation results on Trainer: {'eval_loss': 0.6990557909011841, 'eval_accuracy': 0.395, 'eval_runtime': 1.4971, 'eval_samples_per_second': 267.179, 'eval_steps_per_second': 9.351}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
# These steps are to train and save PEFT model
peft_training_args = TrainingArguments(
    output_dir="./peft_results",
    num_train_epochs=1, 
    per_device_train_batch_size=30,
    per_device_eval_batch_size=30,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./peft_logs",
    logging_steps=10,
    save_strategy="epoch"
)

peft_model_trainer = Trainer(
    model=foundation_model,
    args=peft_training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

peft_model_trainer.train()
peft_model_trainer.save_model("./peft_model")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
# This step is to load the saved PEFT model and evaluate the model
peft_model = AutoModelForSequenceClassification.from_pretrained("./peft_model")

# Evaluate the PEFT model
peft_evaluation_args = TrainingArguments(
    output_dir="./peft_results",
    per_device_eval_batch_size=30,
    logging_dir="./peft_logs",
    logging_steps=10,
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

peft_evaluation_result = peft_trainer.evaluate()

# This step is to show results with PEFT model
print(f"Evaluation results on PEFT Model: {peft_evaluation_result}")

In [ ]:
# This step is to show results of foundatin model and PEFT model
print(f"Please see below to get compariosn details of results between Foundation and PEFT model:")
print(f"Foundation model Evaluation Results: {foundation_evaluation_result['eval_accuracy']}")
print(f"PEFT model Evaluation Results: {peft_evaluation_result['eval_accuracy']}")